In [1]:
import typesense
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

TYPESENSE_KEY = os.environ['typesense_key']

In [3]:
client = typesense.Client({
  'nodes': [{
    'host': 'localhost',
    'port': '8108',
    'protocol': 'http'
  }],
  'api_key': TYPESENSE_KEY,
  'connection_timeout_seconds': 2
})

In [43]:
from pydantic import BaseModel
from typing import Any

class Hit(BaseModel):
    document: dict[str, Any]
    text_match: int
    highlights: list[dict[str, Any]]

class TypesenseSearchResult(BaseModel):
    found: int
    hits: list[Hit]
    page: int
    search_time_ms: int

class TypesenseMultiSearchResult(BaseModel):
    results: list[TypesenseSearchResult]


In [81]:
search_parameters = {
    'q'         : 'naruto',
    'query_by'  : 'name, type, synonyms',
#   'filter_by' : 'num_employees:>100',
#   'sort_by'   : 'num_employees:desc'
}

search_hits = client.collections['entity'].documents.search(search_parameters)
search_result = TypesenseSearchResult.model_validate(search_hits)

In [ ]:
search_hits

In [60]:
q = 'naruto movie'
search_requests = {
    'searches': [
    {
        'collection': 'webpage',
        'q': q,
        'query_by': 'title, url, aka',
    },
    {
        'collection': 'entity',
        'q': q,
        'query_by': 'name, type, description',
    }
    ]
}

# Search parameters that are common to all searches go here
common_search_params =  {
    # 'query_by': 'name',
}

search_hits = client.multi_search.perform(search_requests, common_search_params)
multi_search_result = TypesenseMultiSearchResult.model_validate(search_hits)

In [ ]:
search_hits['results'][1]
# multi_search_result.model_dump()